### Generate SSH credentials

In [3]:
host = "sl1.bsc.es"
user = "bsc23860"
keys_file = "test_keys"


In [4]:
from biobb_remote.ssh_session import SSHCredentials

In [5]:
credentials = SSHCredentials(host=host, userid=user)
credentials.generate_key()
print(credentials.get_public_str())
credentials.save(output_path=keys_file)

ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAABAQDVmrrscgfswlwv4XTmgEukuBG6nkvTKwU8+P7ffNFmP59LFebd4GTKF1EzFmkzQ6Enfw+M7FI5MQ2nG3skNwKYCaFkYoylOhhN7h/CLXCy9tsPncb2yB8tZhIdcXWy/3PSESs0IkAhsg6mN1IJpuqqmem0sMkNyDxDemvCMSt4LcLSmdCyOCZU7j1axeK6PV7/yKodtAbxlqXuRvtCGzNYmHYIExmnpIduNrolSOYZKk9LT40yrwG1gXay10m8swkR8Q0WtyqHZ43ldQcV+2AEqaN4Fm1HQAZ9x+PKclSP+vaA/Qh9r35q/+IrolGzQfx+2TfOFOj5H4gVbyZbxb+H bsc23860@biobb



### Add public key to .ssh/authorized_keys


### Start ssh session and test

In [4]:
from biobb_remote.ssh_session import SSHSession

In [5]:
ssh_session = SSHSession(credentials_path=keys_file)


In [6]:
print(ssh_session.run_command('ls -l')[0])

total 565760
drwxr-xr-x 2 bsc23860 bsc23      4096 Jun 27  2017 bin
-rw-r--r-- 1 bsc23860 bsc23       397 Dec 29 11:11 pb
lrwxrwxrwx 1 bsc23860 bsc23        31 Jul  9  2018 projects -> /slgpfs/projects/bsc23/bsc23860
drwxr-xr-x 2 bsc23860 bsc23      4096 May 31  2018 public_html
lrwxrwxrwx 1 bsc23860 bsc23        30 Jul  9  2018 scratch -> /slgpfs/scratch/bsc23/bsc23860
-rw-r--r-- 1 bsc23860 bsc23 579308027 Sep 27 12:13 uniprot_sprot.dat.gz



### set SLURM queue details

In [7]:
from biobb_remote.slurm import Slurm

In [8]:
keys_file = "test_keys"
job = Slurm()
job.ssh_data.load_from_file(keys_file)

### Check queue status

In [10]:
print(job.check_queue())


             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)



### Input job details

In [11]:
local_data_path = '/home/gelpi/DEVEL/BioExcel/biobb/biobb_remote/test/test_input'
launch_script = local_data_path + '/bsLDH_SLURM.sh'
remote_path ='/home/bsc23/bsc23860/scratch/test_biobb'
queue_settings = 'default'
modules = 'gromacs'
task_file = 'task_file.json'


### Set local input and upload remote data

In [12]:
job.set_local_data_bundle(local_data_path)
job.send_input_data(remote_path)

sending_file: /home/gelpi/DEVEL/BioExcel/biobb/biobb_remote/test/test_input/bsLDH_SLURM.orig.sh -> /home/bsc23/bsc23860/scratch/test_biobb/biobb_43355f27-8ada-4c8f-ba12-791ccf963404/bsLDH_SLURM.orig.sh
sending_file: /home/gelpi/DEVEL/BioExcel/biobb/biobb_remote/test/test_input/bsLDH_md.gro -> /home/bsc23/bsc23860/scratch/test_biobb/biobb_43355f27-8ada-4c8f-ba12-791ccf963404/bsLDH_md.gro
sending_file: /home/gelpi/DEVEL/BioExcel/biobb/biobb_remote/test/test_input/bsLDH_SLURM.sh -> /home/bsc23/bsc23860/scratch/test_biobb/biobb_43355f27-8ada-4c8f-ba12-791ccf963404/bsLDH_SLURM.sh
sending_file: /home/gelpi/DEVEL/BioExcel/biobb/biobb_remote/test/test_input/bsLDH_md.tpr -> /home/bsc23/bsc23860/scratch/test_biobb/biobb_43355f27-8ada-4c8f-ba12-791ccf963404/bsLDH_md.tpr


### Submit job

In [13]:
job.submit(queue_settings, modules, launch_script)

Submitted job 6824164


### Some checks

In [15]:
job.check_job()

'Job 6824164 Running'

In [16]:
stdout, stderr = job.get_logs()
print(stdout)
print(stderr)


hostname sln21
Started.
Running MD...

load impi/2018.3 (PATH, MANPATH, LD_LIBRARY_PATH) 
Set INTEL compilers as MPI wrappers backend 
load FFTW/3.3.8 (PATH, LD_LIBRARY_PATH, LIBRARY_PATH, C_INCLUDE_PATH,
CPLUS_INCLUDE_PATH, HDF5_ROOT, HDF5_DIR) 
load GROMACS/2018.0 (PATH, MANPATH, LD_LIBRARY_PATH, LIBRARY_PATH,
PKG_CONFIG_PATH, C_INCLUDE_PATH, CPLUS_INCLUDE_PATH) 



### Get GROMACS log

In [17]:
print(job.get_remote_file('bsLDH_mdrun.log'))

                       :-) GROMACS - gmx mdrun, 2018 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov  Herman J.C. Berendsen    Par Bjelkmar   
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra    Gerrit Groenhof  
 Christoph Junghans   Anca Hamuraru    Vincent Hindriksen Dimitrios Karkoulis
    Peter Kasson        Jiri Kraus      Carsten Kutzner      Per Larsson    
  Justin A. Lemkul    Viveca Lindahl    Magnus Lundborg   Pieter Meulenhoff 
   Erik Marklund      Teemu Murtola       Szilard Pall       Sander Pronk   
   Roland Schulz     Alexey Shvetsov     Michael Shirts     Alfons Sijbers  
   Peter Tieleman    Teemu Virolainen  Christian Wennberg    Maarten Wolf   
                           and the project leaders:
        Mark Abraham, Berk Hess, Erik Lindahl, and David van der Spoel

Copyright (c) 1991-2000, University of Groningen, The Netherlands.
Copyright (c) 2001-2017, The GROMACS development team at
Uppsala University, Sto

### cancel current job

In [18]:
job.cancel()

Job 6824164 cancelled
Removing remote data for job 6824164


In [23]:
job.check_queue()

'             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)\n'

In [24]:
job.check_job()

Job cancelled by user


'Job 6824164 Cancelled'